# Japanese subset_ main thesis project -ReHo

In [1]:
import pandas as pd
import os
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset, TensorDataset
from torchvision import transforms
from sklearn.model_selection import KFold
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## start of classsification using Reho

In [2]:
control_base_path = '/dbstore/zmohaghegh/Japanese_subset/summary_measures/Control_reho/ReHo_Normalised_z/'
mdd_base_path = '/dbstore/zmohaghegh/Japanese_subset/summary_measures/MDD_reho/ReHo_Normalised_z/'

#define data path for control and MDD seprately

control_files_path = [control_base_path + c for c in os.listdir(control_base_path) if c.startswith('R')]
mdd_files_path = [mdd_base_path + m for m in os.listdir(mdd_base_path) if m.startswith('R')]
print(len(control_files_path), len(mdd_files_path))

251 255


### load data with nan 

In [3]:
#loading data seprately and set a label column for Control=0 and MDD=1 nested list 

control_dataset_reho_zero = [[nib.load(c).get_fdata(),0] for c in control_files_path]
mdd_dataset_reho_zero = [[nib.load(m).get_fdata(),1] for m in mdd_files_path]

##### define numpy array for doing the zero nan

In [4]:
control_reho_zero_nan2= control_dataset_reho_zero
mdd_reho_zero_nan2 = mdd_dataset_reho_zero

print(len(mdd_reho_zero_nan2))
print(control_reho_zero_nan2[0][1]) ### subject label 
print(mdd_reho_zero_nan2[0][0].shape) ### subject nifti -summary measure


mdd_reho_zero_nan3 = np.array(mdd_reho_zero_nan2)
control_reho_zero_nan3 =np.array(control_reho_zero_nan2)


#####control
print(len(control_reho_zero_nan2))
print(control_reho_zero_nan2[0][0].shape)

control_reho_zero_nan3[0][1]

255
0
(91, 109, 91)
251
(91, 109, 91)


/data/zmohaghegh/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':
/data/zmohaghegh/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


0

In [5]:
print(control_reho_zero_nan3.shape[0])
print(control_reho_zero_nan3[0].shape)
print(control_reho_zero_nan3.shape[1])
print(control_reho_zero_nan3[1].shape)
print(control_reho_zero_nan3[0][0].shape)

251
(2,)
2
(2,)
(91, 109, 91)


In [6]:
len(mdd_dataset_reho_zero)

255

In [7]:
len(control_dataset_reho_zero)

251

In [8]:
len(mdd_dataset_reho_zero[0])
len(control_dataset_reho_zero[0])

2

In [9]:
mdd_dataset_reho_zero[0][0].shape
control_dataset_reho_zero[0][0].shape

(91, 109, 91)

In [10]:
control_dataset_reho_zero[190][0][:,34,56]

array([            nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
       -5.60656250e-01, -2.18471512e-01,  2.65964437e-02,  1.17912032e-01,
       -5.86777329e-01, -1.27932799e+00, -5.91229200e-01,  3.55972379e-01,
        4.87391263e-01,  1.03013563e+00,  1.23969424e+00,  1.46534503e+00,
        1.19105673e+00,  9.16325331e-01,  9.76231158e-01,  9.03224111e-01,
        5.72811186e-01, -9.82609694e-04, -6.33052528e-01, -8.04333329e-01,
       -8.95923316e-01, -9.37934101e-01, -7.56355286e-01, -2.29388937e-01,
        5.67567110e-01,  8.79382014e-01,  4.35421288e-01, -2.55543739e-01,
       -2.71465331e-01,  1.12094735e-04,  8.33752751e-01,  7.89981782e-01,
        1.64858356e-01, -1.56577587e-01, -1.83763757e-01, -3.08324248e-01,
       -1.22617386e-01,  

In [11]:
mdd_dataset_reho_zero[100][0][:,34,56]

array([            nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
        1.45227027e+00,  1.58096850e+00,  1.55474269e+00,  1.78304231e+00,
        1.76300514e+00,  1.83052921e+00,  1.73060858e+00,  1.49151731e+00,
        8.38405073e-01,  1.51621774e-01,  5.58712631e-02,  7.96048989e-05,
       -4.80724126e-01, -2.73976892e-01, -4.16711837e-01, -7.36057162e-01,
       -9.18137878e-02,  1.02721065e-01, -2.07152441e-02, -1.40896872e-01,
       -1.06029892e+00, -1.56802857e+00, -1.51698637e+00,  3.41506273e-01,
        6.67224765e-01,  3.86154920e-01,  7.52673447e-02,  5.10709226e-01,
        1.27161467e+00,  7.03470409e-01,  8.21025670e-01,  6.33770823e-01,
       -2.22559988e-01, -1.43178129e+00, -1.59917831e+00, -1.46645176e+00,
       -1.42161059e+00, -

In [12]:
#example control
zero_nan_control_800= np.nan_to_num(control_dataset_reho_zero[100][0],copy=True)
zero_nan_control_800[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.97561115, -0.41167933,  0.45215327,  1.35208154,
        1.81005454,  1.64219189,  0.75563222, -0.85597694, -1.74802983,
       -1.14755046, -0.46927831,  0.1376407 ,  0.55535668, -0.61553514,
       -1.1796025 , -0.36322761, -0.56221068, -0.04582196,  1.29621768,
       -1.07614529, -1.15474641, -1.40290833, -0.10318688,  1.10218644,
       -0.21096501, -0.70668888, -0.83129627,  0.01479594,  0.39268684,
        0.06674624, -0.14645448, -0.20309636,  0.82269102, -1.23148084,
       -1.41650319, -1.12033212, -0.8095482 , -0.38718724, -0.07395672,
       -0.5001781 , -0.74937856, -0.21215707,  0.18034168,  0.32574278,
       -0.03737804,  0.16908635,  0.25720319,  0.50484079,  1.33211768,
        1.79723573,  1.65976012,  1.12066436,  0.90531272,  0.17

In [13]:
#example mdd
zero_nan_mdd_700= np.nan_to_num(mdd_dataset_reho_zero[200][0],copy=True)
zero_nan_mdd_700[:,34,56]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.29152313,  0.44130495,  0.96773052,  0.73992622,
        0.2600117 , -0.74334651, -0.90461767, -0.20272742,  0.47446784,
        0.54080647,  0.2975871 ,  0.25276366, -0.06263041,  0.01617018,
        0.26543203,  0.18063113,  0.06255207,  0.17151089, -0.14120804,
       -0.56722862, -0.31063247, -0.25900012, -0.12108749, -0.64518219,
       -1.4728694 , -1.89632857, -1.14513648, -0.17089775,  0.44840688,
       -0.37902629, -0.96976787, -1.1759299 , -0.70549959, -0.72158879,
       -0.49296591,  0.06177398,  0.81728059,  0.86775333,  1.03077555,
        1.16670609,  1.47919154,  1.60115576,  1.42342389,  0.93247628,
        0.48033294,  0.09809522, -0.33412564,  0.20377555,  0.15091807,
       -0.15625139, -0.15019156, -0.35716426,  0.00880462,  0.26

In [14]:
#zero_nan_control
print(control_reho_zero_nan3.shape)
print(control_reho_zero_nan3[0][0].shape)
print(control_reho_zero_nan3[0][1])
print(control_reho_zero_nan3[100][1])
print(control_reho_zero_nan3[0].shape)
print(type(control_reho_zero_nan3))


for i in range(len(control_dataset_reho_zero)):    
    control_reho_zero_nan3[i][0] =np.nan_to_num(control_dataset_reho_zero[i][0],copy=True)
    control_reho_zero_nan3[i][1] =np.nan_to_num(control_dataset_reho_zero[i][1],copy=True)

(251, 2)
(91, 109, 91)
0
0
(2,)
<class 'numpy.ndarray'>


In [15]:
print(control_reho_zero_nan3.shape)
print(control_reho_zero_nan3[0][0].shape)
print(control_reho_zero_nan3[0][1]) ## mdd label for control=0
print(control_reho_zero_nan3[110][1])
control_reho_zero_nan3[110][0][:,34,56]


(251, 2)
(91, 109, 91)
0
0


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.40634489,  1.16506803,  1.24199009,  1.5856874 ,
        1.90874386,  2.1039741 ,  1.97037923,  1.63948965,  0.0582762 ,
       -0.76295084,  0.14982255,  0.87426162,  0.01851538, -0.89479417,
       -1.10983622, -0.31956229,  0.27067229, -1.15117979, -1.14975691,
       -1.04158092, -1.3259232 , -1.15197194, -0.96472198, -0.29523233,
        0.71875554,  1.56608725,  1.57721782,  0.9202832 ,  0.77033854,
        0.80137569,  1.56249702,  1.45230246,  1.14340329,  0.7310912 ,
        0.4318586 , -0.21386313, -0.24157619, -0.61160183, -0.85414231,
       -0.78928238, -0.11784432, -0.61842167, -0.80912924, -0.50917006,
       -0.21775049, -0.60851109, -0.98969173, -0.21536504,  0.30669811,
        0.5547269 , -0.16132496, -0.08607935,  0.20520933,  0.86

In [16]:
#zero_nan_mdd

print(mdd_reho_zero_nan3.shape)
print(mdd_reho_zero_nan3[0][0].shape)
print(type(mdd_reho_zero_nan3))

for i in range(len(mdd_dataset_reho_zero)):    
    mdd_reho_zero_nan3[i][0] =np.nan_to_num(mdd_dataset_reho_zero[i][0],copy=True)
    mdd_reho_zero_nan3[i][1] =np.nan_to_num(mdd_dataset_reho_zero[i][1],copy=True)

mdd_reho_zero_nan3.shape

(255, 2)
(91, 109, 91)
<class 'numpy.ndarray'>


(255, 2)

In [17]:
print(mdd_reho_zero_nan3.shape)

mdd_reho_zero_nan3.shape
print(mdd_reho_zero_nan3[0][0].shape)
print(mdd_reho_zero_nan3[0].shape)
print(mdd_reho_zero_nan3[0][1])
mdd_reho_zero_nan3[210][0][:,34,56]

(255, 2)
(91, 109, 91)
(2,)
1


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.15539241,  1.09352338,  1.0503757 ,  1.43768322,
        1.51124728,  1.43333316,  0.91985607,  0.08519863,  0.00556017,
       -0.52437329, -0.77414989, -1.12355745, -1.14360178, -1.20118988,
       -0.74124074, -0.30203831, -0.13550924, -0.58327138, -0.92846113,
       -0.50423551, -0.61148393, -0.59212548, -0.61149406, -0.64308512,
       -0.73825824, -0.75959575, -0.96736431, -0.5596289 , -0.66086632,
       -0.96182823, -1.21448779, -1.07356155, -0.8627215 , -0.84807622,
       -1.20948648, -1.15942013, -0.94385028, -0.81308424, -1.16578507,
       -1.2775619 , -1.09162247, -1.41870797, -0.78159875,  0.2320734 ,
        0.72339165,  0.75407511,  0.96091902,  0.93616128,  1.08235204,
        1.20815241,  1.57756925,  1.41936743,  1.42582369,  1.35

### load and add 1 channel to dimension of data 'without nan'

In [18]:
# np.reshape(control_dataset[0][0], (1, 91, 109, 91))

control_reho_4d_zero_nan = [[np.reshape(c[0], (1, 91, 109, 91)), c[1]] for c in control_reho_zero_nan3]
mdd_reho_4d_zero_nan = [[np.reshape(m[0], (1, 91, 109, 91)), m[1]] for m in mdd_reho_zero_nan3]

In [19]:
print(mdd_reho_4d_zero_nan[200][0].shape)
mdd_reho_4d_zero_nan[200][1]

(1, 91, 109, 91)


1

In [20]:
print(control_reho_4d_zero_nan[100][0].shape)
control_reho_4d_zero_nan[100][1]

(1, 91, 109, 91)


0

In [21]:
len(mdd_reho_4d_zero_nan)

255

In [22]:
len(mdd_reho_4d_zero_nan[0])

2

### concat data for control and MDD without nan

In [23]:
dataset_japan_reho_zero_nan= ConcatDataset([control_reho_4d_zero_nan, mdd_reho_4d_zero_nan])

print(dataset_japan_reho_zero_nan[0][0].shape)
print(dataset_japan_reho_zero_nan[400][1])
print(dataset_japan_reho_zero_nan[210][1])


dataset_japan_reho_zero_nan[400][0][0,:,54,45]

#subject number=1500, mdd label,reho value for voxels along axial with y ,z dimention = 54,45

(1, 91, 109, 91)
1
0


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.30531952,  0.3058778 ,  0.32962349,  0.4114182 ,
        0.24398486, -0.0345251 , -0.01915896, -0.16323367, -0.10207997,
       -0.2218703 , -0.73692161, -0.77802765, -0.31200635,  0.11806874,
        0.30682969,  0.3686862 ,  0.40877363,  0.27833337,  0.36288431,
        0.08952609,  0.5410139 , -0.1906241 , -0.4622815 , -0.12508819,
       -0.00884558,  0.09208355, -0.69580549, -1.54331946, -1.02894855,
       -0.24026568, -0.95924526, -2.21315384, -2.32113814, -2.36360312,
       -2.51081634, -1.85549366, -2.15758109, -2.67912054, -1.75750256,
       -0.96856028, -1.57662106, -1.86215401, -1.16792428, -0.79216236,
       -1.09158075, -0.95693052, -1.01139581, -0.64076531, -0.27808905,
        0.27005121,  1.03516877,  0.41657588, -0.41263399, -0.51738447,
       -0.52412993, -0.66663504, -0.68396509, -0.06062748,  0.15

In [24]:
len(dataset_japan_reho_zero_nan)

506

In [25]:
len(dataset_japan_reho_zero_nan[0])

2

In [37]:
print(dataset_japan_reho_zero_nan[200][0].shape) # subject 800 label MDD =0
print(dataset_japan_reho_zero_nan[400][1]) # subject 1500 label MDD =1 

#problem solved : mdd_zero_nan label have problem : instead of 1 integer we have an array (91,109,91)

print(dataset_japan_reho_zero_nan[0][1])
print(dataset_japan_reho_zero_nan[400][1])
dataset_japan_reho_zero_nan[400][0].shape

(1, 91, 109, 91)
1
0
1


(1, 91, 109, 91)

### Network architecture

In [ ]:
#Define a Convolutional Neural Network : BASED ON https://www.biorxiv.org/content/10.1101/2019.12.17.879346v1

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.downsample = nn.AvgPool3d(2, stride=2, padding=0)
        
        self.CNNlayer = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=3, stride=1),
            nn.ELU(),
            nn.Conv3d(64, 16, kernel_size=3, stride=1),
            nn.ELU(),
            nn.MaxPool3d(2)
        )
        
        self.flat1 = nn.Linear(160000, 16)   
        self.flat2 = nn.Linear(16, 1)
            
    def forward(self, x):
        x=self.downsample(x)
        #print(f'avg-pool: {x.size()}\n----------')
        #print(f'number of nan in this layer = {torch.isnan(x).sum()}')
        
        x=self.CNNlayer(x)
        #print(f'convolution1+2+maxpool: {x.size()} \n----------')
        
        x=x.reshape(x.size(0), -1)
        #print(f'reshape after cnn: {x.size()}\n----------')
        
        x=F.elu(self.flat1(x))
        #print(f'fully-connected1: {x.size()}\n----------')
                    
        x=self.flat2(x)
        #print(f'fully-connected2: {x.size()}\n----------')
        
        return x

### lunch wandb

In [40]:
import wandb
!wandb login 390734ff44d817dbba59927d4eb542e564627b3b

wandb: Appending key for api.wandb.ai to your netrc file: /data/zmohaghegh/.netrc


### Train and Test (without CV)

In [ ]:
### spilit data to train-test and validation

dataset_size= len(dataset_ukbb_reho_zero_nan)
train_count = int(0.7 * dataset_size) 
valid_count= int(0.2 * dataset_size)
test_count = dataset_size - train_count -valid_count

print(dataset_size)
print(train_count)
print(valid_count)
print(test_count)

In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score,confusion_matrix

In [ ]:
wandb.init(project='mdd-japanese-reho')  

#define the newtwork 
network = ConvNet()
network = network.double()

#set hyper parameter
learning_rate= 0.001
num_epochs = 10
BATCH_SIZE = 10
best_acc= None

# Define a Loss function 
loss_function = nn.BCEWithLogitsLoss()  

# create the optimizer
optimizer = optim.SGD(network.parameters(), momentum=0.9, lr = learning_rate, weight_decay=1e-3)

# load data splits
train_dataset, test_dataset ,valid_dataset = torch.utils.data.random_split(dataset_japan_reho_zero_nan, (train_count, test_count,valid_count))

train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)  
test_dataset_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=BATCH_SIZE, shuffle=False)
valid_dataset_loader  = torch.utils.data.DataLoader(valid_dataset , batch_size=BATCH_SIZE, shuffle=True)


#dataloaders = {'train': train_dataset_loader, 'test': test_dataset_loader, 'valid': valid_dataset_loader}

network.train()
# start training loop:
for epoch in range(0, num_epochs):
    print(f'*********Starting epoch {epoch+1}')
    
    train_loss = 0
    total =0 
    correct = 0
    
    # train model/network 
    for i, data in enumerate(train_dataset_loader, 0):
        #print(f'train {i}')
            
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
            
        # zero the gradient buffers
        optimizer.zero_grad()
            
        # forward pass
        outputs = network(inputs)
    
        outputss = outputs.squeeze(1) #### [10,1] ---> [10]
        
        #nloss requires a 1-D tensor of class indices of the target (outputs in your case) but not one-hot vectors of the target.
        #The shape of the target should be [mini_batch_size], 
        #but in your case it happens to be [mini_batch_size,num_classes].
        
        predicted = outputss.data > 0.0 # the loss function contain also a sigmoid layer <0 = false >0 =true
        
        labels = labels.double()
        
        #calcuate loss/error
        loss = loss_function(outputss, labels)
              
        # backward pass
        loss.backward()
            
        # Does the update , gradient descent
        optimizer.step() 
            
        #loss_list.append(loss.item())
        train_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f'acc_train :{100 * correct/total}')
    print (f'loss_train = {train_loss/total}')
    
    wandb.log({ "epoch": epoch ,"train_Loss": train_loss/total, "trian_acc" :100 * correct /total })

    
    print('Training process has finished') 

    print('starting validation :')

    network.eval()

    valid_loss=0
    total=0
    correct=0

    with torch.no_grad():
        for i, data in enumerate(valid_dataset_loader, 0):
            #print(f'valid {i}')
            inputs, lables = data

            outputs = network(inputs)

            outputss=outputs.squeeze(1) #[10,1] ---> [10]
            lables=lables.double()

            # prediction 
            predicted = outputss.data > 0.0 # the loss function contain also a sigmoid layer <0 = false >0 =tru

            loss = loss_function(outputss, lables)

            valid_loss += loss.item()
            total += lables.size(0)
            correct += (predicted == lables).sum().item()

        print(f'acc_valid :{100 * correct/total}')
        print (f'loss_valid = {valid_loss/total}')

        wandb.log({ "validation_acc" :100 * correct /total, "validation_Loss": valid_loss/total })

        current_valid_acc =100 * correct/total
        if not best_acc or best_acc < current_valid_acc:
            best_acc = current_valid_acc
            
            print('Saving trained model.')
            PATH_best = './model-japanese-best-acc-reho.pth'
            torch.save(network.state_dict(), PATH_best)
        print('validation process has finished.')

### Test the network using the test data without CV

In [ ]:
network.load_state_dict(torch.load(PATH_best))

test_loss = 0
correct = 0
total= 0

print('Starting testing')

network.eval()

with torch.no_grad():
    for i, data in enumerate(test_dataset_loader, 0):
        #print(f'test {i}')
        
        inputs, lables = data
        lables = lables.double() # loss function get only double not Long
        
        outputs = network(inputs)
        #print(outputs.size())
        
        outputss = outputs.squeeze(1) #[10,1] ---> [10]
        #print(outputss.size())
        #print(lables.size())
    
        predicted = outputss.data > 0.0 # the loss function contain also a sigmoid layer <0 = false >0 =true
        
        loss = loss_function(outputss, lables)
        
        test_loss += loss.item()
        total += lables.size(0)
        correct += (predicted == lables).sum().item()
        
    wandb.log({ "test_Accuracy": 100 * correct /total  , "test_Loss": test_loss/total })

    print(f'test_loss= {test_loss/total}')
    print(f'test_acc= {100 * correct / total}')

## Cross validation loop 

In [ ]:
#from sklearn.model_selection import train_test_split, StratifiedKFold

#train_dataset_cv, test_dataset_cv = train_test_split(dataset_ukbb_reho_zero_nan, test_size=0.20, train_size=0.8,shuffle=True)
#print(len(train_dataset_cv))
#print(len(test_dataset_cv))

In [39]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import statistics

In [ ]:
#imgage[0].shape
#image.shape[0]

In [41]:
len(dataset_japan_reho_zero_nan)

506

In [43]:
train_valid_folds

KFold(n_splits=2, random_state=None, shuffle=True)

In [49]:
train_ids

array([  0,   1,   2,   3,   4,   5,   6,   8,   9,  11,  12,  13,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  37,  38,  39,  40,  41,  42,
        43,  45,  46,  48,  49,  50,  52,  53,  54,  55,  56,  59,  60,
        61,  63,  64,  65,  66,  68,  69,  70,  71,  73,  74,  75,  76,
        78,  79,  80,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        96,  97,  98,  99, 101, 104, 105, 106, 107, 108, 109, 110, 111,
       112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 144, 145, 146, 147, 149, 150, 152, 153, 154, 155,
       157, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171,
       172, 173, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186,
       188, 189, 191, 192, 193, 194, 195, 196, 197, 199, 200, 201, 202,
       203, 204, 205, 207, 208, 209, 210, 211, 212, 213, 216, 21

In [47]:
train_valid_folds.split(dataset_japan_reho_zero_nan[train_ids][0])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### train and validation and test loop CV

In [50]:
wandb.init(project='mdd-japanese-reho')  

k_folds = 5
kfold_results = {}

# StratifiedKFold instead of KFOLD for balanced number of class for each fold
kfold = KFold(n_splits=k_folds, shuffle=True)  

#Hyper parameter
num_epochs = 20
batch_size = 1
learning_rate= 0.00001


#Define a Loss function 
loss_function = nn.BCEWithLogitsLoss()

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset_japan_reho_zero_nan)):
    best_loss_cv= None
    print(test_ids)
    
    print(f"FOLD {fold}\n--------------------------------")
    
    train_valid_folds = KFold(n_splits=2, shuffle=True)
    len(train_valid_folds)
    
    train_ids_new, valid_ids = list(train_valid_folds.split(dataset_japan_reho_zero_nan[train_ids]))[0]
    
    # Create training,testing and validation datasets
    train_data = dataset_ukbb_reho_zero_nan[train_ids_new]
    valid_data = dataset_ukbb_reho_zero_nan[valid_ids]
    test_data = dataset_ukbb_reho_zero_nan[test_ids]
    
    # Sample elements randomly from a given list of ids,
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids_new)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)
    
    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_subsampler)
    valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, sampler=valid_subsampler)
    test_loader  = torch.utils.data.DataLoader(test_data , batch_size=batch_size, shuffle=False)

    
    #define network
    network = ConvNet()
    network = network.double()
    
    # create our optimizer
    optimizer = optim.SGD(network.parameters(), momentum=0.9, lr = learning_rate, weight_decay=1e-3)
    
    # in the training loop:
    
    network.train() # prepare model for training
    
    for epoch in range(0, num_epochs):
        print(f'*********Starting epoch {epoch+1}')
        
        train_loss_cv = 0
        total =0
        correct=0

        # train model/network 
        for i, data in enumerate(train_loader, 0):
            #print(f'train {i}')
            
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            
            # zero the gradient buffers
            optimizer.zero_grad()
            
            # forward pass
            outputs = network(inputs)
            
            # print(outputs.size)
            outputss=outputs.squeeze(1) #### [10,1] ---> [10]
            
            # prediction 
            predicted = outputss.data > 0.0 # the loss function contain also a sigmoid layer <0 = false >0 =true
            
            labels=labels.double()
            
            #calcuate loss/error
            loss = loss_function(outputss, labels)
            
            # backward pass
            loss.backward()
            
            # Does the update , gradient descent
            optimizer.step() 
            
            correct += (predicted == labels).sum().item()
            train_loss_cv += loss.item()
            total += labels.size(0)
            
        #wandb.log({ "epoch_cv": epoch , "train_Loss_cv": train_loss_cv/total, "train_acc_cv": 100 * correct / total })
        
        print(f'train loss= {train_loss_cv/total}')
        print(f'train Acc= {100 * correct / total}')
        
        
        print('Training process has finished.')
        print('Starting testing')

        # validate the network using the validation data, for this epoch
        correct= 0
        total = 0
        valid_loss_cv=0
        
        network.eval()

        with torch.no_grad():
            for i, data in enumerate(valid_loader, 0):
                #print(f'test {i}')

                inputs, lables = data

                outputs = network(inputs)

                outputss=outputs.squeeze(1) #[10,1] ---> [10]
                lables=lables.double()

                # prediction 
                predicted = outputss.data > 0.0 # the loss function contain also a sigmoid layer <0 = false >0 =true

                loss = loss_function(outputss, lables)

                valid_loss_cv += loss.item()
                total += lables.size(0)
                correct += (predicted == lables).sum().item()

            #wandb.log({ f'validation_acc_fold_{fold}': 100 * correct /total, f'validation_Loss_fold_{fold}': valid_loss_cv/total})
            
            current_valid_loss_cv = 100 * correct /total
            
            print(f'valid_acc :{100 * correct /total}')
            print(f'valid_loss: {valid_loss_cv/total}')
                  
            if not best_loss_cv or best_loss_cv < current_valid_loss_cv:
                best_loss_cv = current_valid_loss_cv
                best_network=network
                
                print('Saving best valid -trained model.')
                path_best_loss_reho = f'/data/zmohaghegh/TempStats_3D-CNN/best_model_reho_japanese/model-japanese_best-reho-fold-{fold}.pth'
                torch.save(network.state_dict(), path_best_loss_reho )
                
            print('validation process has finished')
        
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    
        
    kfold_results[fold] = 100.0 * (correct / total)
    
    print(f"K-FOLD CROSS VALIDATION RESULTS FOR japanese reho {k_folds} FOLDS\n--------------------------------")
    _sum = 0.0

    for key, value in kfold_results.items():
        print(f'Fold {key}: {value} %')
        _sum += value

    print(f'Average: {_sum/len(kfold_results.items())} %')
    
    
    #start testing 
    print('start testing...')
    
    #network.load_state_dict(torch.load(path_fold))

    test_loss_cv=0
    total = 0
    correct=0

    F1_labels=[]
    F1_pred=[]
    F1_scores=[]
    bal_accs=[]

    best_network.eval() # preoare model for test and evaluation

    with torch.no_grad():
        
        #print('Start testing CV...')
        for i, data in enumerate(test_loader, 0):
            #print(f'test {i}')

            inputs, lables = data

            outputs = best_network(inputs)

            lables=lables.double()
            outputss=outputs.squeeze(1) #[10,1] ---> [10]

            #do the prediction 
            predicted = outputss.data > 0.0  #the loss function contain also a sigmoid layer <0 = false >0 =true

            #calculate loss
            loss = loss_function(outputss, lables)

            test_loss_cv += loss.item()
            correct += (predicted == lables).sum().item()
            total += lables.size(0)

            if i==0:
                F1_labels=lables.int().numpy()
                F1_pred=predicted.int().numpy()
            else:
                F1_labels= np.concatenate((F1_labels, lables.int().numpy()))
                F1_pred = np.concatenate((F1_pred, predicted.int().numpy()))
                    
    acc_fold = accuracy_score(F1_labels, F1_pred)
    bal_acc_fold= balanced_accuracy_score(F1_labels, F1_pred)
    F1_Score_fold = f1_score(F1_labels, F1_pred, average='weighted')
    
    print(f'test_Acc_CV": {acc_fold}')
    print(f'Balanced ACC CV ReHo :{bal_acc_fold}')
    print(f'F1_score CV ReHo :{F1_Score_fold}')

    
    #tn, fp, fn, tp = confusion_matrix(F1_labels, F1_pred).ravel()
    
    F1_scores.append(F1_Score_fold)
    bal_accs.append(bal_acc_fold)
        
    #PLotting Confusion matrix and ROC curve
        
    #conf_matrix = confusion_matrix(F1_labels, F1_pred)
    #conf_matrix_display = ConfusionMatrixDisplay(conf_matrix).plot()
        
    #fp_rate, tp_rate, threshold = roc_curve(F1_labels, F1_pred)
    #ROC_display = RocCurveDisplay(fpr=fp_rate, tpr=tp_rate).plot()
       

    #wandb.log({ "test_balanced_Acc_CV":bal_acc_fold , "test_F1_score_CV": F1_Score_fold,'test_Acc': 100 * correct /total })
        

F1_score_avg= sum(F1_scores)/len(F1_scores)
F1_score_std= statistics.pstdev(F1_scores)
bal_acc_avg = 100 * (sum(bal_accs)/len(bal_accs))
bal_acc_std = 100* statistics.pstdev(bal_accs)

print(f' Average F1_score japan ReHo = { F1_score_avg}')
print(f'standard deviation :{F1_score_std}')
print(f' Average Balance ACC japan ReHo = {bal_acc_avg}')
print(f'standard deviation :{bal_acc_std}')

[  5  17  20  22  33  40  48  50  55  57  62  65  69  75  79  84  85  88
  93  96  99 101 119 128 129 142 148 152 155 156 164 167 172 177 180 186
 191 194 197 198 204 214 219 221 222 223 236 237 240 247 248 256 267 269
 270 271 273 274 276 279 280 289 304 314 316 331 336 340 341 351 356 357
 358 360 362 367 381 383 384 390 392 400 408 413 414 416 418 429 432 435
 444 451 464 466 477 482 488 493 496 501 502 504]
FOLD 0
--------------------------------


TypeError: object of type 'KFold' has no len()

### results of train and validation for CV

In [ ]:
print(f"K-FOLD CROSS VALIDATION RESULTS FOR japanese reho {k_folds} FOLDS\n--------------------------------")
_sum = 0.0

for key, value in kfold_results.items():
    print(f'Fold {key}: {value} %')
    _sum += value

print(f'Average: {_sum/len(kfold_results.items())} %')

### Test loop for Cross validation 

In [ ]:
wandb.init(project='mdd-japanese-reho')  

#Sample elements randomly 

test_loader  = torch.utils.data.DataLoader(test_dataset_cv , batch_size=batch_size, shuffle=False)

bal_acc_fold=[]
F1_score_fold=[]

for k in np.arange(5): 

    print(f'Start TEST FOR FOLD {k}')

    path_fold = f'/data/zmohaghegh/TempStats_3D-CNN/best_model_reho_japanese/model-japanese_best-reho-fold-{k}.pth'

    network.load_state_dict(torch.load(path_fold))

    test_loss_cv=0
    total = 0
    correct=0

    F1_labels=[]
    F1_pred=[]

    network.eval() # preoare model for test and evaluation

    with torch.no_grad():
        #print('Start testing CV...')
        for i, data in enumerate(test_loader, 0):
            #print(f'test {i}')

            inputs, lables = data

            outputs = network(inputs)

            lables=lables.double()
            outputss=outputs.squeeze(1) #[10,1] ---> [10]

            #do the prediction 
            predicted = outputss.data > 0.0  #the loss function contain also a sigmoid layer <0 = false >0 =true

            #calculate loss
            loss = loss_function(outputss, lables)

            test_loss_cv += loss.item()
            correct += (predicted == lables).sum().item()
            total += lables.size(0)

            if i==0:
                F1_labels=lables.int().numpy()
                F1_pred=predicted.int().numpy()
            else:
                F1_labels= np.concatenate((F1_labels, lables.int().numpy()))
                F1_pred = np.concatenate((F1_pred, predicted.int().numpy()))
            
        #wandb.log({ 'test_Acc': 100 * correct /total  , "test_Loss": test_loss_cv/total })
        
        acc = accuracy_score(F1_labels, F1_pred)
        bal_acc= balanced_accuracy_score(F1_labels, F1_pred)

        F1_Score = f1_score(F1_labels, F1_pred, average='weighted')
        tn, fp, fn, tp = confusion_matrix(F1_labels, F1_pred).ravel()
    
        F1_score_fold.append(F1_Score)
        bal_acc_fold.append(bal_acc)
        
        #PLotting Confusion matrix and ROC curve
        
        conf_matrix = confusion_matrix(F1_labels, F1_pred)
        conf_matrix_display = ConfusionMatrixDisplay(conf_matrix).plot()
        
        fp_rate, tp_rate, threshold = roc_curve(F1_labels, F1_pred)
        ROC_display = RocCurveDisplay(fpr=fp_rate, tpr=tp_rate).plot()
       

        wandb.log({ "test_balanced_Acc_CV": bal_acc , "test_F1_score_CV": F1_Score })
        
        print(f'test_Acc_CV": {100 * correct /total}')
        print(f'F1_score CV ReHo :{F1_Score}')
        print(f'Balanced ACC CV ReHo :{bal_acc}')
        print(f'Loss CV ReHo : {test_loss_cv/total}')

In [ ]:
F1_score_avg= sum(F1_score_fold)/len(F1_score_fold)
F1_score_std= statistics.pstdev(F1_score_fold)
bal_acc_avg = 100 * (sum(bal_acc_fold)/len(bal_acc_fold))
bal_acc_std = 100* statistics.pstdev(bal_acc_fold)

print(f' Average F1_score japan ReHo = { F1_score_avg}')
print(f'standard deviation :{F1_score_std}')
print(f' Average Balance ACC japan ReHo = {bal_acc_avg}')
print(f'standard deviation :{bal_acc_std}')